In [43]:
#导入数据时增加一个维度变为（10505，1，30，200），都要增加吗？
#batchsize=16的时候，是怎么放入模型的？第一个卷积输入通道为1，是（1500，1，30，1024）中的1吗？
#（1500，1，30，1024）叫3维数据还是4？不是二维卷积码？
#（8，1，1，64）解释？
#实例化类的时候，传入的参数是——init的？
#标签制作：不用独热编码的区别？
#打印的print("block1", x.shape)指的是这一层的数据的形状吗
#十折交叉验证修改一下？不知道对不对 chatgpt生成的
#classes_num=2去除的时候，EEGNet.__init__() missing 1 required positional argument: 'classes_num'


In [44]:
import random
import torch
from sklearn.model_selection import KFold
import math
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

In [45]:
# 设置GPU使用方式
# 获取GPU列表
gpus = [torch.cuda.device(i) for i in range(torch.cuda.device_count())]

if gpus:
    try:
        # 设置每个GPU为增长式占用
        for gpu in gpus:
            torch.cuda.set_per_process_memory_fraction(1.0, gpu)  # 可根据需要调整占用比例
    except RuntimeError as e:
        # 打印异常
        print(e)

In [46]:
class SCNNB(nn.Module):
    def __init__(self):
        super(SCNNB, self).__init__()

        self.conv1 = nn.Conv2d(1,32,kernel_size=(3,3))
        self.BN1 = nn.BatchNorm2d(32,False)
        self.pooling1 = nn.MaxPool2d(2,2)

        self.conv2 = nn.Conv2d(32,64,kernel_size=(3,3))
        self.BN2 = nn.BatchNorm2d(64,False)
        self.pooling2 = nn.MaxPool2d(2,2)

        self.fc1 = nn.Linear(1600,1280)
        self.fc2 = nn.Linear(1280,10)
        
    def forward(self,x):
        #print('第零层',x.shape)
        x = self.pooling1(F.relu(self.BN1(self.conv1(x))))
        #print('第一层',x.shape)
        x = self.pooling2(F.relu(self.BN2(self.conv2(x))))
        #print('第二层',x.shape)
        x = x.reshape(-1,1600)
        #print('第三层',x.shape)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, 0.5)
        x = self.fc2(x)  # 将 dropout 放在 fc1 后
        #print(x.shape)
        return x

In [47]:
import torch
from torchvision import datasets, transforms

# 假设数据集的路径
data_path = 'D:\code\classifier_data'

# 定义数据转换
transform = transforms.Compose([
    transforms.ToTensor(),  # 将图片转换为 Tensor
    transforms.Normalize((0,), (255,))  # 归一化
])

# 加载 MNIST 数据集
mnist_dataset = datasets.MNIST(root=data_path, train=True, transform=transform, download=False)

# 将数据和标签分离
data = mnist_dataset.data
labels = mnist_dataset.targets

# 将数据和标签转换为 Tensor
data_all = data.unsqueeze(1)  # 添加通道维度 (N, C, H, W)，其中 C = 1
label_all = labels.long()  # 确保标签为 long 类型

In [48]:
# 创建十折交叉验证
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

# 保存结果的列表
historys = []
test_pred = []
test_real = []
accuracy, precision, recall, f1score = [], [], [], []
batchsz = 16

# 进行十折交叉验证
for fold, (train_ind, test_ind) in enumerate(kfold.split(data_all, label_all)):
    print('fold号:', fold + 1)

    # 每一折验证前都要打乱训练集样本顺序
    n = len(train_ind)
    A = np.linspace(0, n - 1, n, dtype=int)
    random.shuffle(A)

    # 构建训练集、验证集、测试集
    epoch_train = data_all[train_ind[A[:int(0.8 * n)]]]
    epoch_val = data_all[train_ind[A[int(0.8 * n):]]]
    epoch_test = data_all[test_ind]
    label_train = label_all[train_ind[A[:int(0.8 * n)]]]
    label_val = label_all[train_ind[A[int(0.8 * n):]]]
    label_test = label_all[test_ind]

    # 转换为Tensor并创建DataLoader
    train_dataset = TensorDataset(torch.tensor(epoch_train, dtype=torch.float32), torch.tensor(label_train, dtype=torch.long))
    val_dataset = TensorDataset(torch.tensor(epoch_val, dtype=torch.float32), torch.tensor(label_val, dtype=torch.long))
    
    train_loader = DataLoader(train_dataset, batch_size=batchsz, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batchsz, shuffle=False)

    # 选择、创建模型
    model = SCNNB()
    #print(model)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)

    # 配置模型训练
    criterion = nn.CrossEntropyLoss()  # 使用交叉熵损失
    optimizer = optim.Adam(model.parameters(), lr=1e-4)

    # 开始训练模型
    print('开始训练!!')
    history = {'train_loss': [], 'val_loss': []}

    for epoch in range(10):  # 训练100个epoch
        model.train()
        running_loss = 0.0

        # 训练阶段
        for inputs, targets in train_loader:
            #print(f"训练批次输入数据形状: {inputs.shape}")
            inputs = inputs.to(device)
            
            targets = targets.to(device)
            optimizer.zero_grad()
            
            outputs = model(inputs)
            targets = targets.view(-1) 
            #targets=targets.view(-1,1).float()
          

            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        avg_train_loss = running_loss / len(train_loader)
        history['train_loss'].append(avg_train_loss)

        # 验证阶段
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs = inputs.to(device)
            
                targets = targets.to(device)
                outputs = model(inputs)
                #targets=targets.view(-1,1).float()
                targets = targets.view(-1) 
                loss = criterion(outputs, targets)
                val_loss += loss.item()

        avg_val_loss = val_loss / len(val_loader)
        history['val_loss'].append(avg_val_loss)

    # 保存训练记录
    historys.append(history)

    # 计算、保存测试结果
    model.eval()
    with torch.no_grad():
        pred_test = model(torch.tensor(epoch_test, dtype=torch.float32).to(device))
        pred_test = torch.argmax(pred_test, dim=1).cpu().numpy()

    # 保存预测结果和真实结果
    test_pred.append(pred_test)
    test_real.append(label_test)

    # 计算准确率，精确率，召回率，F1评分
    print(label_test.shape)
    print(pred_test.shape)
    acc = accuracy_score(label_test, pred_test)
    pre = precision_score(label_test, pred_test, average='macro')
    rec = recall_score(label_test, pred_test, average='macro')
    f1 = f1_score(label_test, pred_test, average='macro')
    
    accuracy.append(acc)
    precision.append(pre)
    recall.append(rec)
    f1score.append(f1)

    print(f"$$ 测试集准确率为 accuracy: {acc}")
    print(f"$$ 测试集精确率为 precision: {pre}")
    print(f"$$ 测试集召回率为 recall: {rec}")
    print(f"$$ 测试集 F1 评分为 f1_score: {f1}")

# 将每一折 history 中误差结果保存（训练集和测试集，用于反映训练过程）    
loss_train = []
loss_val = []
for history_s in historys:
    loss_val.append(history_s['val_loss'])
    loss_train.append(history_s['train_loss'])

fold号: 1


C:\Users\FJQ\AppData\Local\Temp\ipykernel_10496\4261917605.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_dataset = TensorDataset(torch.tensor(epoch_train, dtype=torch.float32), torch.tensor(label_train, dtype=torch.long))
C:\Users\FJQ\AppData\Local\Temp\ipykernel_10496\4261917605.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  val_dataset = TensorDataset(torch.tensor(epoch_val, dtype=torch.float32), torch.tensor(label_val, dtype=torch.long))


开始训练!!


C:\Users\FJQ\AppData\Local\Temp\ipykernel_10496\4261917605.py:97: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pred_test = model(torch.tensor(epoch_test, dtype=torch.float32).to(device))


torch.Size([6000])
(6000,)
$$ 测试集准确率为 accuracy: 0.9886666666666667
$$ 测试集精确率为 precision: 0.9886313197143574
$$ 测试集召回率为 recall: 0.988711851990729
$$ 测试集 F1 评分为 f1_score: 0.9886513308316663
fold号: 2
开始训练!!


C:\Users\FJQ\AppData\Local\Temp\ipykernel_10496\4261917605.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_dataset = TensorDataset(torch.tensor(epoch_train, dtype=torch.float32), torch.tensor(label_train, dtype=torch.long))
C:\Users\FJQ\AppData\Local\Temp\ipykernel_10496\4261917605.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  val_dataset = TensorDataset(torch.tensor(epoch_val, dtype=torch.float32), torch.tensor(label_val, dtype=torch.long))


KeyboardInterrupt: 